# Model Interpretability Notebook using Gradient-based Attribution Methods

In [ ]:
import os

import torch

from odyssey.data.tokenizer import ConceptTokenizer
from odyssey.interp.attribution import Attribution
from odyssey.interp.utils import get_type_id_mapping
from odyssey.models.model_utils import (
    load_config,
    load_finetune_data,
    load_finetuned_model,
)
from odyssey.utils.utils import seed_everything

### Arguments


In [ ]:
MODEl_TYPE = "cehr_bert"
MODEL_PATH = "/ssd003/projects/aieng/public/odyssey/bert_checkpoints/best.ckpt"
TOKENIZER_PATH = None
CONFIG_DIR = "../models/configs"
DATA_DIR = "/ssd003/projects/aieng/public/odyssey/new_data"
VOCAB_DIR = "/ssd003/projects/aieng/public/odyssey/vocab"
CODES_DIR = "/ssd003/projects/aieng/public/odyssey/new_data/codes_dict"
SEQUENCE_FILE = "patient_sequences_512_mortality.parquet"
ID_FILE = "dataset_2048_mortality.pkl"
VALID_SCHEME = "few_shot"
LABEL_NAME = "label_mortality_1month"
NUM_FINETUNE_PATIENTS = "20000"
WITH_TASKS = False

SEED = 42
BATCH_SIZE = 32
N_STEPS = 100
MAX_LEN = 512

### Config Setup


In [ ]:
seed_everything(SEED)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
config = load_config(CONFIG_DIR, MODEl_TYPE)
finetune_config = config["finetune"]
pre_model_config = config["model"]
fine_model_config = config["model_finetune"]

### Load Tokenizer

In [ ]:
if TOKENIZER_PATH is not None:
    tokenizer = ConceptTokenizer.load(TOKENIZER_PATH)
else:
    tokenizer = ConceptTokenizer(data_dir=VOCAB_DIR)
    tokenizer.fit_on_vocab(with_tasks=WITH_TASKS)

### Load Data

In [ ]:
_, test_data = load_finetune_data(
    DATA_DIR,
    SEQUENCE_FILE,
    ID_FILE,
    VALID_SCHEME,
    NUM_FINETUNE_PATIENTS,
)
test_data.rename(columns={LABEL_NAME: "label"}, inplace=True)

In [ ]:
# Take a sample data point
test_data_sample = test_data.head(20)
test_data_sample

### Load Model

In [ ]:
model = load_finetuned_model(
    MODEl_TYPE,
    MODEL_PATH,
    tokenizer=tokenizer,
    pre_model_config=pre_model_config,
    fine_model_config=fine_model_config,
    device=device,
)
model

### Get Attributions


In [ ]:
gradient_attr = Attribution(
    test_data_sample,
    model,
    tokenizer,
    device,
    type_id_mapping=get_type_id_mapping(),
    max_len=MAX_LEN,
    batch_size=BATCH_SIZE,
    n_steps=N_STEPS,
    codes_dir=CODES_DIR,
)

In [ ]:
# Average attribution for tokens
token_avg = gradient_attr.average_tokens_attr()
token_avg

In [ ]:
# Average attribution for embeddings
embedding_avg = gradient_attr.average_embeddings_attr()
embedding_avg

### Visualize Attributions

In [ ]:
vis = gradient_attr.visualize_integrated_gradients(
    max_rows=10,
    task_name="post-discharge mortality",
)

In [ ]:
vis = gradient_attr.visualize_expected_gradients(
    max_rows=10,
    num_baselines=10,
    task_name="post-discharge mortality",
)